In [ ]:
import os

key_path = "user_id.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

# Optional: confirm it's set
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

In [ ]:
import os
os.environ["CPL_VSIL_USE_TEMP_FILE_FOR_RANDOM_WRITE"] = "YES"
# --- Optional: faster temp dir if you have SSD space ---
os.environ["TMPDIR"] = "/mnt/e"  # or /mnt/nvme/temp_gdal on Linux

In [ ]:
import os
from google.cloud import storage

# 2️⃣ Initialize GCS client
client = storage.Client.from_service_account_json(key_path)

# 3️⃣ List buckets (to verify connection)
for bucket in client.list_buckets():
    print("✅ Found bucket:", bucket.name)

In [ ]:
bucket_name = "remote_sensing_saas"
bucket = client.bucket(bucket_name)

blobs = list(bucket.list_blobs(prefix=""))  # list all files
for blob in blobs:  # show only first 10
    if '.tif' in blob.name:
        print(blob.name)

In [ ]:
import time
from osgeo import gdal

# --- CONFIG ---
src = "/mnt/d/KORINDO_DRONE/BLOK 1/ORTHO_PT_STL_KALBAR_Blok 1_TIFF.tif"      # Input GeoTIFF in GCS
dst = "/vsigs/remote_sensing_saas/01-korindo/00-drone/ORTHO_PT_STL_KALBAR_Blok 1_TIFF_COG.tif" # Output COG in GCS

# --- START TIMER ---
start = time.time()

# --- DEFINE PROGRESS CALLBACK ---
def progress_callback(complete, message, unknown):
    percent = int(complete * 100)
    print(f"\rProgress: {percent}% | {message}", end="")

# --- RUN TRANSLATE ---
print(f"🚀 Starting GDAL COG translation...\nInput: {src}\nOutput: {dst}\n")

translate_options = [
    "OVERVIEWS=IGNORE_EXISTING",
    "BIGTIFF=YES", 
    "COMPRESS=ZSTD",
    "LEVEL=22",
    "PREDICTOR=2",
    "INTERLEAVE=BAND",
    "NUM_THREADS=10"
]

result = gdal.Translate(
    dst,
    src,
    format="COG",
    creationOptions=translate_options,
    callback=progress_callback
)

# --- END TIMER ---
end = time.time()
elapsed = end - start

if result is not None:
    print(f"\n✅ Done! COG successfully created in {elapsed:.2f} seconds.")
else:
    print(f"\n❌ GDAL translate failed after {elapsed:.2f} seconds.")


In [ ]:
### FOR LOWER SPEC
import os
import time
from osgeo import gdal

# --- GDAL Cloud Storage Fix (Crucial!) ---
os.environ["CPL_VSIL_USE_TEMP_FILE_FOR_RANDOM_WRITE"] = "YES"

# --- 1. SET TEMP DIRECTORY ---
# CRITICAL: You must MANUALLY CREATE this folder first!
# Go to C:\ and create a new folder named "temp".
os.environ["CPL_TMPDIR"] = r"c:\temp" 

# --- OPTIMIZE FOR LOW RAM (8GB) ---
gdal.SetConfigOption('GDAL_CACHEMAX', '1024')

# --- CONFIG ---
src = r"C:\Users\treeo-workspace\Documents\BLOK 4\ORTHO_PT_STL_KALBAR_Blok 4_TIFF.tif"
dst = r"c:\temp\ORTHO_PT_STL_KALBAR_Blok 4_TIFF_COG.tif"

# --- START TIMER ---
start = time.time()

# --- DEFINE PROGRESS CALLBACK ---
def progress_callback(complete, message, unknown):
    percent = int(complete * 100)
    print(f"\rProgress (local creation): {percent}% | {message}      ", end="", flush=True)

# --- 2. RUN TRANSLATE ---
print(f"🚀 Starting GDAL COG translation...\nInput: {src}\nOutput: {dst}\n")

translate_options = [
    "OVERVIEWS=IGNORE_EXISTING",
    "BIGTIFF=YES", 
    "COMPRESS=ZSTD",
    "LEVEL=9",
    "PREDICTOR=2",
    "INTERLEAVE=BAND",
    "NUM_THREADS=6" 
]

result = gdal.Translate(
    dst,
    src,
    format="COG",
    creationOptions=translate_options,
    callback=progress_callback
)

# Ensure we print a newline after the progress bar
print() 

# --- 3. THIS IS THE CORRECTED PART ---
if result is not None:
    print("✅ Local COG creation complete.")
    print("⏳ Forcing upload to Google Cloud Storage... (This may take a while)")
    
    # --- THIS IS THE FIX ---
    # Force GDAL to write all data and close the file handle.
    # This blocks the script until the upload is 100% finished.
    result.FlushCache()  
    result = None        # This line triggers the actual upload
    # --- END FIX ---
    
    # NOW we can safely stop the timer
    end = time.time()
    elapsed = end - start
    print(f"✅ Upload complete! Total time: {elapsed:.2f} seconds.")
    print("\n--- Running verification... ---")
    
    # --- 4. IMMEDIATE VERIFICATION ---
    gdal.PushErrorHandler('CPLQuietErrorHandler') 
    ds_verify = gdal.Open(dst)
    gdal.PopErrorHandler() 

    if ds_verify is not None:
        print("✅✅✅ SUCCESS! File is confirmed in the bucket.")
        ds_verify = None
    else:
        print("❌❌❌ FAILURE! Verification step failed. File not found in bucket.")

else:
    # This means gdal.Translate failed locally
    end = time.time()
    elapsed = end - start
    print(f"❌ GDAL translate failed during *local* creation after {elapsed:.2f} seconds.")

In [ ]:
### UPLOADING TO GCS
import os
from google.cloud import storage
from tqdm.notebook import tqdm  # <-- Import tqdm for Jupyter

def upload_file_to_gcs_with_progress(bucket_name, source_file_path, destination_blob_name):
    """
    Uploads a local file to GCS with a progress bar in Jupyter.

    Args:
        bucket_name (str): The name of your GCS bucket.
        source_file_path (str): The local path to the file you want to upload.
        destination_blob_name (str): The name you want the file to have in GCS.
    """
    # 1. Instantiate the GCS client
    storage_client = storage.Client()

    # 2. Get the target bucket
    try:
        bucket = storage_client.bucket(bucket_name)
    except Exception as e:
        print(f"Error getting bucket '{bucket_name}': {e}")
        return

    # 3. Create a new blob (file) in the bucket
    blob = bucket.blob(destination_blob_name)

    # 4. Get the total file size for the progress bar
    try:
        file_size = os.path.getsize(source_file_path)
    except FileNotFoundError:
        print(f"Error: The file {source_file_path} was not found.")
        return
    except Exception as e:
        print(f"Error getting file size: {e}")
        return

    # 5. Upload the file using upload_from_file and the tqdm wrapper
    try:
        # Open the file in binary-read mode
        with open(source_file_path, "rb") as f:
            
            # Create the tqdm wrapper
            # - 'f' is the file object
            # - "read" is the method to wrap
            # - total is the total file size
            # - unit="B" and unit_scale=True create human-readable sizes (e.g., MB)
            with tqdm.wrapattr(f, "read", total=file_size, desc="Uploading", unit="B", unit_scale=True, unit_divisor=1024) as file_with_progress:
                
                # Use upload_from_file, passing the wrapped object and the total size
                blob.upload_from_file(file_with_progress, size=file_size)

        print(f"\n✅ Upload complete: {source_file_path} to {bucket_name}/{destination_blob_name}.")

    except Exception as e:
        print(f"\nError during upload: {e}")

# --- How to use the function ---
# (Assuming 'bucket_name' and 'dst' are defined in your Jupyter notebook)

MY_BUCKET_NAME = bucket_name  # <-- Make sure 'bucket_name' is defined in a cell above
dst = dst                     # <-- Make sure 'dst' (your file path) is defined in a cell above

GCS_DESTINATION_NAME = f"01-korindo/00-drone/{os.path.basename(dst)}"

###Call the new, correct function
upload_file_to_gcs_with_progress(MY_BUCKET_NAME, dst, GCS_DESTINATION_NAME)